_unchecked_

# CNTK 302 B 部分: 使用神经和甘斯的图像超分辨率

_unchecked_

<p><strong>贡献的</strong><a href="https://www.linkedin.com/in/borna-vukorepa-32a35283/">博尔纳 Vukorepa</a>2017年10月30日</p>
<h2>介绍</h2>
<p><p>此笔记本下载图像数据, 并训练可以从较低分辨率图像创建更高分辨率图像的模型。用户应该在这之前完成教程 CNTK 302A, 这样他们就能熟悉解决它的超分辨率问题和方法。<b>单图像超分辨率 (SISR)</b>问题的目标是, 通过某种因素对给定的图像进行高级化, 同时保持尽可能多的图像细节, 而不会使图像模糊不清。</p>
<p>为了训练我们的模型, 我们需要准备数据。不同的模型可能需要不同的训练集。</p>
<p>我们将使用<b>伯克利 Segmetation 数据集 (bsd)</b>。它包含300图像, 我们将下载, 然后准备培训的模型, 我们将使用。图像尺寸为 481 x 321 和 321 x 481。</p></p>

In [1]:
# Import the relevant modules to be used later
import urllib
import re
import os
import numpy as np
from PIL import Image
import sys
import cntk as C

try:
    from urllib.request import urlretrieve, urlopen
except ImportError: 
    from urllib import urlretrieve, urlopen

_unchecked_

### 选择笔记本运行时环境设备/设置

将设备设置为测试环境的 cpu/gpu。如果您的计算机上有 CPU 和 GPU, 则可以选择切换设备。默认情况下, 我们选择最佳的可用设备。

In [2]:
# Select the right target device when this notebook is being tested:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))

_unchecked_

<p>有两种运行模式:<ul>
<li><i>快速模式:</i> <code>isFast</code>设置为 True。这是笔记本的默认模式, 这意味着我们在有限的数据上进行较少的迭代或培训/测试。这确保了笔记本的功能正确性, 虽然所生产的模型远非完成的培训所能产生的。</li>
<li><i>慢速模式</i>: 我们建议用户在用户熟悉笔记本内容后将此标志设置为 False, 并希望从运行笔记本的过程中获得更长时间的洞察力, 并使用不同的参数进行培训。</li>
</ul></p>

<p><b>注意</b>如果 <code>isFlag</code> 设置为 False, 则笔记本电脑将在启用 GPU 的计算机上花费数天时间。您可以通过将 <code>NUM_MINIBATCHES</code> 设置为较小的数字来尝试较少的迭代, 以牺牲生成图像的质量为代价。也可以尝试减少 <code>MINIBATCH_SIZE</code> 。</p>

In [3]:
isFast = True

_unchecked_

## 数据下载

数据不存在于. zip、. gz 或类似的包中。它位于此[链接](https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300/html/images/plain/normal/color/)的文件夹中。因此, 我们将使用正则表达式来查找所有图像名称, 并将它们一个一个地下载到目标文件夹中。但是, 如果数据已在测试模式下下载或运行, 则笔记本将使用缓存的数据。

In [4]:
# Determine the data path for testing
# Check for an environment variable defined in CNTK's test infrastructure
envvar = 'CNTK_EXTERNAL_TESTDATA_SOURCE_DIRECTORY'
def is_test(): return envvar in os.environ

if is_test():
    test_data_path_base = os.path.join(os.environ[envvar], "Tutorials", "data")
    test_data_dir = os.path.join(test_data_path_base, "BerkeleySegmentationDataset")
    test_data_dir = os.path.normpath(test_data_dir)

# Default directory in a local folder where the tutorial is run
data_dir = os.path.join("data", "BerkeleySegmentationDataset")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
#folder with images to be evaluated
example_folder = os.path.join(data_dir, "example_images")
if not os.path.exists(example_folder):
    os.makedirs(example_folder)

#folders with resulting images
results_folder = os.path.join(data_dir, "example_results")
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [5]:
def download_data(images_dir, link):
    #Open the url
    images_html = urlopen(link).read().decode('utf-8')
    
    #looking for .jpg images whose names are numbers
    image_regex = "[0-9]+.jpg"
    
    #remove duplicates
    image_list = set(re.findall(image_regex, images_html))
    print("Starting download...")
    
    num = 0
    
    for image in image_list:
        num = num + 1
        filename = os.path.join(images_dir, image)
        
        if num % 25 == 0:
            print("Downloading image %d of %d..." % (num, len(image_list)))
        if not os.path.isfile(filename):
            urlretrieve(link + image, filename)
        else:
            print("File already exists", filename)
    
    print("Images available at: ", images_dir)

_unchecked_

现在我们只需要用适当的参数调用这个函数。这可能需要几分钟的时间。

In [6]:
#folder for raw images, before preprocess
images_dir = os.path.join(data_dir, "Images")
if not os.path.exists(images_dir):
    os.makedirs(images_dir)
    
#Get the path for pre-trained models and example images
if is_test():
    print("Using cached test data")
    models_dir = os.path.join(test_data_dir, "PretrainedModels")
    images_dir = os.path.join(test_data_dir, "Images")
else:
    models_dir = os.path.join(data_dir, "PretrainedModels")
    if not os.path.exists(models_dir):
        os.makedirs(models_dir)
    
    images_dir = os.path.join(data_dir, "Images")
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)
        
    #link to BSDS dataset
    link = "https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300/html/images/plain/normal/color/"

    download_data(images_dir, link)
        
print("Model directory", models_dir)
print("Image directory", images_dir)

Starting download...
Images available at:  data/BerkeleySegmentationDataset/Images
Model directory data/BerkeleySegmentationDataset/PretrainedModels
Image directory data/BerkeleySegmentationDataset/Images


_unchecked_

<h2>数据准备</h2>
<p><p>此数据集仅包含300图像, 这对于超分辨率训练来说是不够的。这个想法是将图像分割成 64 x 64 补丁, 这将增加训练数据。函数 <code>prep_64</code> 完全正确。</p>
<p>一旦我们选择了一些 64 x 64 补丁, 我们降低它的因素 2, 然后高档它的一个因素2再次使用次插值。这将给我们一个模糊的版本的原始补丁。在本教程的一些方法中, 想法将是学习将模糊的路径变成清晰的模型。我们将把模糊的补丁放入一个文件夹和正常补丁到另一个。他们将作为 minibatch 的培训来源。我们也将在这里采样一些测试图像。</p>
<p>在处理每个补丁后, 我们移动42像素下/右 (所以有一些重叠之间的补丁), 只要我们可以。此预处理可能需要几分钟时间。</p></p>

In [7]:
#extract 64 x 64 patches from BSDS dataset
def prep_64(images_dir, patch_h, patch_w, train64_lr, train64_hr, tests):
    if not os.path.exists(train64_lr):
        os.makedirs(train64_lr)
    
    if not os.path.exists(train64_hr):
        os.makedirs(train64_hr)
    
    if not os.path.exists(tests):
        os.makedirs(tests)
    
    k = 0
    num = 0
    
    print("Creating 64 x 64 training patches and tests from:", images_dir)
    
    for entry in os.listdir(images_dir):
        filename = os.path.join(images_dir, entry)
        img = Image.open(filename)
        rect = np.array(img)
    
        num = num + 1
        
        if num % 25 == 0:
            print("Processing image %d of %d..." % (num, len(os.listdir(images_dir))))
        
        if num % 50 == 0:
            img.save(os.path.join(tests, str(num) + ".png"))
            continue
    
        x = 0
        y = 0
    
        while(y + patch_h <= img.width):
            x = 0
            while(x + patch_w <= img.height):
                patch = rect[x : x + patch_h, y : y + patch_w]
                img_hr = Image.fromarray(patch, 'RGB')
                
                img_lr = img_hr.resize((patch_w // 2, patch_h // 2), Image.ANTIALIAS)
                img_lr = img_lr.resize((patch_w, patch_h), Image.BICUBIC)
                
                out_hr = os.path.join(train64_hr, str(k) + ".png")
                out_lr = os.path.join(train64_lr, str(k) + ".png")
                
                k = k + 1
                
                img_hr.save(out_hr)
                img_lr.save(out_lr)
                
                x = x + 42
            y = y + 42
    print("Done!")

_unchecked_

<p>我们将下载一个 pretrained CNTK VGG19 模型, 稍后将用于培训<b>SRGAN</b>模型。该模型将对 224 x 224 图像进行操作, 因此我们需要为将 112 x 112 图像转换为 224 x 224 图像的模型准备培训数据。<p>我们使用类似的推理在这里用 <code>prep_224</code> 扩充训练数据。我们将选择 224 x 224 补丁, 并将它们缩小到 112 x 112 补丁。唯一的区别是, 现在我们也旋转补丁, 以增加培训样本的数量, 因为随着补丁的大小增加到 224 x 224, 我们的训练集减少了需要使用, 以增加培训数据集更多。像以前一样, 224 x 224 补丁进入一个文件夹和 112 x 112 补丁进入另一个。</p>

In [8]:
#extract 224 x 224 and 112 x 112 patches from BSDS dataset
def prep_224(images_dir, patch_h, patch_w, train112, train224):
    if not os.path.exists(train112):
        os.makedirs(train112)
    
    if not os.path.exists(train224):
        os.makedirs(train224)

    k = 0
    num = 0
    
    print("Creating 224 x 224 and 112 x 112 training patches from:", images_dir)
    
    for entry in os.listdir(images_dir):
        filename = os.path.join(images_dir, entry)
        img = Image.open(filename)
        rect = np.array(img)
        
        num = num + 1
        if num % 25 == 0:
            print("Processing image %d of %d..." % (num, len(os.listdir(images_dir))))
    
        x = 0
        y = 0
    
        while(y + patch_h <= img.width):
            x = 0
            while(x + patch_w <= img.height):
                patch = rect[x : x + patch_h, y : y + patch_w]
                img_hr = Image.fromarray(patch, 'RGB')
                
                img_lr = img_hr.resize((patch_w // 2, patch_h // 2), Image.ANTIALIAS)
                
                for i in range(4):
                    out_hr = os.path.join(train224, str(k) + ".png")
                    out_lr = os.path.join(train112, str(k) + ".png")
                
                    k = k + 1
                
                    img_hr.save(out_hr)
                    img_lr.save(out_lr)
                
                    img_hr = img_hr.transpose(Image.ROTATE_90)
                    img_lr = img_lr.transpose(Image.ROTATE_90)
                
                x = x + 64
            y = y + 64
    print("Done!")

_unchecked_

<p>我们使用最初下载的图像, 运行 <code>prep</code> 函数有足够的参数, 并生成大约300图像从伯克利的分割数据, 根据多少去测试集。</p>

<p>在文件夹<b>train64_LR</b>中, 我们将有大约 20000 64 x 64 模糊图像补丁, 将用于培训。它们的原始对应项将位于文件夹<b>train64_HR</b>中。</p>

<p>在文件夹<b>train224</b>中, 我们将有大约12000原始 224 x 224 补丁。它们的缩减 112 x 112 对等方将位于<b>train112</b>。稍后可用于测试的图像将位于<b>测试</b>中。</p>

In [9]:
#blurry 64x64 destination
train64_lr = os.path.join(data_dir, "train64_LR")

#original 64x64 destination
train64_hr = os.path.join(data_dir, "train64_HR")

#112x112 patches destination
train112 = os.path.join(data_dir, "train112")

#224x224 pathes destination
train224 = os.path.join(data_dir, "train224")

#tests destination
tests = os.path.join(data_dir, "tests")

#prep
prep_64(images_dir, 64, 64, train64_lr, train64_hr, tests)
prep_224(images_dir, 224, 224, train112, train224)

Creating 64 x 64 training patches and tests from: data/BerkeleySegmentationDataset/Images
Processing image 25 of 300...
Processing image 50 of 300...
Processing image 75 of 300...
Processing image 100 of 300...
Processing image 125 of 300...
Processing image 150 of 300...
Processing image 175 of 300...
Processing image 200 of 300...
Processing image 225 of 300...
Processing image 250 of 300...
Processing image 275 of 300...
Processing image 300 of 300...
Done!
Creating 224 x 224 and 112 x 112 training patches from: data/BerkeleySegmentationDataset/Images
Processing image 25 of 300...
Processing image 50 of 300...
Processing image 75 of 300...
Processing image 100 of 300...
Processing image 125 of 300...
Processing image 150 of 300...
Processing image 175 of 300...
Processing image 200 of 300...
Processing image 225 of 300...
Processing image 250 of 300...
Processing image 275 of 300...
Processing image 300 of 300...
Done!


_unchecked_

<h2>数据插图</h2>
<p><p>在下图中, 我们将显示数据的外观。大图像是300伯克利的分割数据集图像之一。</p>
<p>较大的补丁 (以红色突出显示) 是一对 224 x 224 和 112 x 112 pathces 的示例。它们可以分别在文件夹 train224 和 train112 中找到。</p>
<p>较小的补丁 (以蓝色高亮显示) 是一个 64 x 64 清晰和模糊的补丁的例子。它们可以分别在文件夹 train64_HR 和 train64_LR 中找到。</p></p>

In [10]:
from IPython.display import Image
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/train_data.png")

_unchecked_

现在我们准备开始训练了。

_unchecked_

<h2>使用的模型</h2>
<p><p>在本教程的剩余部分, 我们将展示如何使用<b>认知工具箱 (CNTK)</b>创建几个用于解决<b>SISR</b>问题的深层卷积网络。</p>
<p>我们将尝试几个不同的模型, 看看我们能得到什么结果。我们将训练的模型是:</p>
<ul>
<li><b>VDSR</b><a href="http://cv.snu.ac.kr/research/VDSR/VDSR_CVPR2016.pdf">(非常深的超级分辨率)</a></li>
<li><b>DRRN</b><a href="http://cvlab.cse.msu.edu/pdfs/Tai_Yang_Liu_CVPR2017.pdf">(深层递归剩余网络)</a></li>
<li><b>SRResNet</b><a href="http://openaccess.thecvf.com/content_cvpr_2017_workshops/w12/papers/Lim_Enhanced_Deep_Residual_CVPR_2017_paper.pdf">(超分辨率剩余网络)</a></li>
<li><b>SRGAN</b><a href="http://openaccess.thecvf.com/content_cvpr_2017/papers/Ledig_Photo-Realistic_Single_Image_CVPR_2017_paper.pdf">(超级分辨率生成对抗性网络)</a></li>
</ul>
<p>每个模型都有其背后的一些关键的想法, 它们将在本笔记本中更详细地讨论。我们将从<b>VDSR</b>模型开始。</p></p>

_unchecked_

## VDSR 超分辨率模型

_unchecked_

<p><b>VDSR</b>模型背后的关键思想是<b>剩余学习</b>。而不是预测低分辨率图像的高分辨率图像, 我们首先使用一些廉价的方法, 如次插值, 我们的启动图像。然后, 我们预测所谓的 "残差图像", 即高分辨率图像和我们获得的图像在第一步次插值的差异。Therefeore, 模型中和预测图像中的所有中间值都是小的, 比较稳定。最后的结果是通过增加预测值和廉价的提升图像得到的。您可以在<a href="https://arxiv.org/pdf/1511.04587.pdf">此处</a>看到相关的纸张。</p>

<p>请注意, 该模型的作用是已插值的低分辨率图像, 这是已经提升的因子为 2, 所以输入和输出图像的尺寸是相同的。训练集必须以支持这个想法的方式来准备。</p>

<p>卷积网络由交替卷积层和 recitfying 线性单元 (ReLUs) 组成。总共有19卷积层。最后一个卷积之后没有 ReLU 层。</p>

In [11]:
#Figure 1
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/vdsr_architecture.png")

_unchecked_

<p>上面的<b>图 1</b>取自纸张<a href="https://arxiv.org/pdf/1511.04587.pdf">使用非常深的卷积网络的精确图像超分辨率</a>。它显示<b>VDSR</b>模型的体系结构。从插值低分辨率图像 (劳资) 出发, 对残差图像进行预测。其中的大多数值为零或非常小。添加残余图像和劳资给我们的高分辨率图像的预测。</p>

_unchecked_

## 培训配置

_unchecked_

<p>我们将使用 minibatches 的大小64。如果要使<i>CUDA 内存</i>不足错误, 请降低 minibatch 大小。通过<b>伯克利 Segementation 数据集 (BSDS300)</b> (文件夹<b>train64_LR</b>和<b>train64_HR</b>) 编写的 64 x 64 输入补丁程序对模型进行了培训。</p>

In [12]:
#training configuration
MINIBATCH_SIZE = 16 if isFast else 64
NUM_MINIBATCHES = 200 if isFast else 200000

# Ensure the training and test data is generated and available for this tutorial.
# We search in two locations for the prepared Berkeley Segmentation Dataset.
data_found = False

for data_dir in [os.path.join("data", "BerkeleySegmentationDataset")]:
    train_hr_path = os.path.join(data_dir, "train64_HR")
    train_lr_path = os.path.join(data_dir, "train64_LR")
    if os.path.exists(train_hr_path) and os.path.exists(train_lr_path):
        data_found = True
        break
        
if not data_found:
    raise ValueError("Please generate the data by completing the first part of this notebook.")
    
print("Data directory is {0}".format(data_dir))

#folders with training data (high and low resolution images) and paths to map files
training_folder_HR = os.path.join(data_dir, "train64_HR")
training_folder_LR = os.path.join(data_dir, "train64_LR")
MAP_FILE_Y = os.path.join(data_dir, "train64_HR", "map.txt")
MAP_FILE_X = os.path.join(data_dir, "train64_LR", "map.txt")

#image dimensions
NUM_CHANNELS = 3
IMG_H, IMG_W = 64, 64
IMAGE_DIMS = (NUM_CHANNELS, IMG_H, IMG_W)

Data directory is data/BerkeleySegmentationDataset


_unchecked_

### 创建 minibatch 源

下面的功能将创建映射文件的文件夹与图像补丁 (既高, 低分辨率的图像)。地图文件将用于创建 minibatches 的培训数据。此功能将用于为本教程中的所有模型创建映射文件。

In [13]:
# create a map file from a flat folder
import cntk.io.transforms as xforms

def create_map_file_from_flatfolder(folder):
    file_endings = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
    map_file_name = os.path.join(folder, "map.txt")
    with open(map_file_name , 'w') as map_file:
        for entry in os.listdir(folder):
            filename = os.path.join(folder, entry)
            if os.path.isfile(filename) and os.path.splitext(filename)[1] in file_endings:
                tempName = '/'.join(filename.split('\\'))
                tempName = '/'.join(tempName.split('//'))
                tempName = '//'.join(tempName.split('/'))
                map_file.write("{0}\t0\n".format(tempName))
    return map_file_name

_unchecked_

下面的函数将从上面提到的地图文件中创建 minibatch 源。稍后, 我们将使用它来创建低分辨率和高分辨率图像的 minibatch 源。此函数将用于为所有模型创建 minibatch 源。

In [14]:
# creates a minibatch source for training or testing
def create_mb_source(map_file, width, height, num_classes = 10, randomize = True):
    transforms = [xforms.scale(width = width,  height = height, channels = NUM_CHANNELS, interpolations = 'linear')]
    return C.io.MinibatchSource(C.io.ImageDeserializer(map_file, C.io.StreamDefs(
        features = C.io.StreamDef(field = 'image', transforms = transforms),
        labels = C.io.StreamDef(field = 'label', shape = num_classes))), randomize = randomize)

_unchecked_

### VDSR 体系结构

_unchecked_

<p>对于<b>VDSR</b>模型, 我们将只使用卷积层和 ReLU 层, 总是一个接一个地堆积, 除了最后一个在卷积层后没有 ReLU 层的时候。</p>

<p>我们将有19卷积层。每个神经元都有大小为 3 x 3 的接收字段, 每个层都包含64过滤器, 但最后一个有三过滤器除外。它们是结果的 RGB 表示形式。下面是构建此体系结构的函数定义。</p>

In [15]:
def VDSR(h0):
    print('Generator input shape: ', h0.shape)
    
    with C.layers.default_options(init = C.he_normal(), activation = C.relu, bias = False):
        model = C.layers.Sequential([
            C.layers.For(range(18), lambda :
                C.layers.Convolution((3, 3), 64, pad = True)),
            C.layers.Convolution((3, 3), 3, activation = None, pad = True)
        ])
    
    return model(h0)

_unchecked_

### 计算图

_unchecked_

<p>我们的模型采用模糊 (插值低分辨率) 64 x 64 图像。它们是从高分辨率的对应物中获得的, 它们将它们缩小, 然后推广回原来的大小。该模型将对其进行评估, 并预测在低分辨率图像中添加的残差图像。该结果与原始高分辨率图像之间的欧氏距离是我们希望最小化的损失。输出也是 64 x 64, 但更少的模糊和接近原始。</p>

<p><code>real_X</code>存储低分辨率图像和 <code>real_Y</code> 存储原始高分辨率图像。</p>

<p>为了优化, 我们使用亚当学习者。学习率从 <code>0.1</code> 开始, 逐渐递减 <code>10</code> 。</p>

In [16]:
#computation graph
def build_VDSR_graph(lr_image_shape, hr_image_shape, net):
    input_dynamic_axes = [C.Axis.default_batch_axis()]
    real_X = C.input(lr_image_shape, dynamic_axes = input_dynamic_axes, name = "real_X")
    real_Y = C.input(hr_image_shape, dynamic_axes = input_dynamic_axes, name = "real_Y")

    real_X_scaled = real_X/255
    real_Y_scaled = real_Y/255

    genG = net(real_X_scaled)
    
    #Note: this is where the residual error is calculated and backpropagated through Generator
    g_loss_G = IMG_H * IMG_W * C.reduce_mean(C.square(real_Y_scaled - real_X_scaled - genG)) / 2.0
    
    G_optim = C.adam(g_loss_G.parameters, lr = C.learning_rate_schedule(
        [(1, 0.1), (1, 0.01), (1, 0.001), (1, 0.0001)], C.UnitType.minibatch, 50000),
                   momentum = C.momentum_schedule(0.9), gradient_clipping_threshold_per_sample = 1.0)

    G_G_trainer = C.Trainer(genG, (g_loss_G, None), G_optim)

    return (real_X, real_Y, genG, real_X_scaled, real_Y_scaled, G_optim, G_G_trainer)

_unchecked_

<h2>培训</h2>
<p><p>现在我们准备训练我们的模型。利用以上的功能和培训配置, 建立 minibatch 源和计算图。然后我们采取 minibatches 一个一个, 慢慢更新我们的模型。迭代次数 (minibatches) 取决于 <code>isFast</code> 是否为 True。<p>
<p>请注意, <code>train</code> 函数接受模型体系结构、低分辨率图像的维度和计算图函数。这将使我们能够对所有模型使用相同的函数, 除了<b>SRGAN</b>模型, 我们还需要训练鉴别器。</p>
<p>训练与 isFast 设置为真实可能需要大约10分钟。</p></p>

In [17]:
#training
def train(arch, lr_dims, hr_dims, build_graph):
    create_map_file_from_flatfolder(training_folder_LR)
    create_map_file_from_flatfolder(training_folder_HR)
    
    print("Starting training")

    reader_train_X = create_mb_source(MAP_FILE_X, lr_dims[1], lr_dims[2])
    reader_train_Y = create_mb_source(MAP_FILE_Y, hr_dims[1], hr_dims[2])
    real_X, real_Y, genG, real_X_scaled, real_Y_scaled, G_optim, G_G_trainer = build_graph(lr_image_shape = lr_dims,
                                                                                           hr_image_shape = hr_dims, net = arch)
    
    print_frequency_mbsize = 50
   
    pp_G = C.logging.ProgressPrinter(print_frequency_mbsize)

    input_map_X = {real_X: reader_train_X.streams.features}
    input_map_Y = {real_Y: reader_train_Y.streams.features}
    
    for train_step in range(NUM_MINIBATCHES):
        
        X_data = reader_train_X.next_minibatch(MINIBATCH_SIZE, input_map_X)
        batch_inputs_X = {real_X: X_data[real_X].data}
        
        Y_data = reader_train_Y.next_minibatch(MINIBATCH_SIZE, input_map_Y)
        batch_inputs_X_Y = {real_X : X_data[real_X].data, real_Y : Y_data[real_Y].data}

        G_G_trainer.train_minibatch(batch_inputs_X_Y)
        pp_G.update_with_trainer(G_G_trainer)
        G_trainer_loss = G_G_trainer.previous_minibatch_loss_average
    
    return (G_G_trainer.model, real_X, real_X_scaled, real_Y, real_Y_scaled)

In [18]:
VDSR_model, real_X, real_X_scaled, real_Y, real_Y_scaled = train(VDSR, IMAGE_DIMS, IMAGE_DIMS, build_VDSR_graph)

Starting training
Generator input shape:  (3, 64, 64)
 Minibatch[   1-  50]: loss = 1583398986.511333 * 800;
 Minibatch[  51- 100]: loss = 3.904708 * 800;
 Minibatch[ 101- 150]: loss = 3.996467 * 800;
 Minibatch[ 151- 200]: loss = 4.015535 * 800;


_unchecked_

<p>现在该模型可用于超分辨任意图像。请记住, VDSR 模型操作 64 x 64 图像获得的次插值, 并返回一个 64 x 64 的图像, 比开始一个更清楚。它如何能够超分辨任意大小的图像？这个想法是先把目标图像次 ** 插值, 然后去补丁, 以清除他们与我们的模型, 并呈现产生的图像。</p>

<p><p>还要记住, 损失函数是基于缩放的图像 (像素值介于0和1之间), 因此, 在将预测的残差添加到低分辨率图像之前, 必须将其缩减255。如果某些像素变为负值或超过 255, 则在保存之前分别将它们返回到0和255。</p>
<p><b>图 2</b>是我们使用此模型可以得到的结果的示例。左边是次插值的作用, 右边是我们的模型应用的结果。</p></p>

In [19]:
#Figure 2
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/vdsr_small_test.png")

_unchecked_

<h2>DRRN 超分辨率模型</h2>
<p><p><b>DRRN</b>代表深层的 "递归" 剩余网络。严格地说, 模型不是递归的, 因为没有向后流动的数据。它与已描述的<b>VDSR</b>模型非常相似, 因为它还使用<b>残余学习</b>的概念, 这意味着我们只预测残余图像, 即插值低分辨率图像与高分辨率图像。与在<b>VDSR</b>之前一样, 添加这两项将给我们最终的结果, 希望有很多高频细节。
您可以在<a href="http://cvlab.cse.msu.edu/pdfs/Tai_Yang_Liu_CVPR2017.pdf">此处</a>看到相关的纸张。显示模型体系结构的图像也来自该文件。</p>
<p><b>DRRN</b>仅在模型体系结构中与<b>VDSR</b>不同, 因此我们只需要定义一个构建该体系结构的新函数。<b>DRRN</b>体系结构由几个<b>递归块</b>组成。每个递归块由两个卷积层组成。每一个都是批规范化层和 ReLU 层。</p>
<p><b>图 3</b>取自上述文件, 并演示如何依次堆叠递归块 (分别为一、二和三块)。</p></p>

In [20]:
#Figure 3
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/recblock_architecture.png")

_unchecked_

<p><b>图 4</b>也来自同一篇文章, 显示了具有两个递归块的<b>DRRN</b>模型体系结构。</p>

In [21]:
#Figure 4
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/drnn_architecture.png")

_unchecked_

<h3>DRRN 建筑学和训练</h3>
<p><p>我们将有9递归块。在最后一个卷积之后将没有 ReLU 层。每个神经元都有大小为 3 x 3 的接收字段, 每个层都包含128过滤器, 但最后一个有三过滤器除外。它们是结果的 RGB 表示形式。</p><p>之后, 我们只需要调用 <code>train</code> 函数 <code>DRRN</code> 作为参数, 因为其他所有内容都与<b>VDSR</b>相同, 包括训练数据和计算图。</p>
<p>训练与 isFast 设置为真实可能需要大约25分钟。</p></p>

In [22]:
#basic DRRN block
def DRRN_basic_block(inp, num_filters):
    c1 = C.layers.Convolution((3, 3), num_filters, init = C.he_normal(), pad = True, bias = False)(inp)
    c1 = C.layers.BatchNormalization(map_rank = 1)(c1)
    return c1

def DRRN(h0):
    print('Generator input shape: ', h0.shape)
    
    with C.layers.default_options(init = C.he_normal(), activation = C.relu, bias = False):
        h1 = C.layers.Convolution((3, 3), 128, pad = True)(h0)
        h2 = DRRN_basic_block(h1, 128)
        h3 = DRRN_basic_block(h2, 128)
        h4 = h1 + h3
    
        for _ in range(8):
            h2 = DRRN_basic_block(h4, 128)
            h3 = DRRN_basic_block(h2, 128)
            h4 = h1 + h3
        
        h_out = C.layers.Convolution((3, 3), 3, activation = None, pad = True)(h4)

        return h_out

In [23]:
DRRN_model, real_X, real_X_scaled, real_Y, real_Y_scaled = train(DRRN, IMAGE_DIMS, IMAGE_DIMS, build_VDSR_graph)

Starting training
Generator input shape:  (3, 64, 64)
 Minibatch[   1-  50]: loss = 2977.761893 * 800;
 Minibatch[  51- 100]: loss = 166.646252 * 800;
 Minibatch[ 101- 150]: loss = 141.640209 * 800;
 Minibatch[ 151- 200]: loss = 166.618866 * 800;


_unchecked_

<p>现在, 这个模型可以用于超解析任意图像, 就像我们在<b>VDSR</b>部分中描述的那样。正如我们已经提到的, 我们将评估我们的模型, 并最终创建过滤器。</p>

<p><b>图 5</b>是我们使用此模型可以得到的结果的示例。和以前一样, 左边是次插值的效果, 右边是我们的模型应用的结果。更多的迭代可以给我们带来更多的改进。质量低于<b>VDSR</b> (从培训错误中可见), 因为我们只培训了很少数量的迭代。<b>DRRN</b>和<b>VDSR</b>将显示类似的性能, 当我们应用经过大量迭代训练的模型时。</p>

In [24]:
#Figure 5
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/drnn_small_test.png")

_unchecked_

<h2>SRResNet 超分辨率模型</h2>
<p><p>与前两个模型不同, <b>SRResNet</b>不使用剩余学习, 并且在预处理中没有推广。在模型评估过程中, 利用卷积转置 (也称为反褶积) 提升了图像。现在的训练集是不同的, 因为输入和输出有不同的维度。我们的模型将采取 112 x 112 图像和输出 224 x 224 图像。这是因为<b>SRGAN</b>以后需要的<b>CNTK VGG19</b>模型的 pretrained 版本严格操作 224 x 224 图像。</p>
<p>模型体系结构的基础是<b>剩余块</b>。每个残块有两个卷积层, 每个都有一个在第一个 (PReLU) 之后的参数整流线性单元的批规范化 (BN) 层。卷积层有 3 x 3 接收字段, 其中每一个包含64过滤器。图象 resoultion 在模型的结尾附近增加, 比增加它在开始时是较不复杂的。您可以在<a href="https://arxiv.org/pdf/1609.04802.pdf">此处</a>看到相关的纸张。</p>
<p>本文中的<b>图 6</b>以更详细的方式显示了模型体系结构。<code>k</code>是接收字段大小, <code>n</code> 是层中的筛选器数, <code>s</code> 是步幅。</p></p>

In [25]:
#Figure 6
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/srresnet_architecture.PNG")

_unchecked_

<h3>培训配置</h3>
<p><p>由于模型的操作不同的图像大小和输入尺寸不同于输出 (记住, 推广是在模型内部完成的, 而不是预处理), 我们必须改变训练配置。我们遵循的纸张和使用 minibatch 大小的16。我们使用为该模型准备的培训集 (文件夹<b>train224</b>和<b>train112</b>), 因此需要更改培训文件夹和映射文件路径。</p></p>

In [26]:
#training configuration
MINIBATCH_SIZE = 8 if isFast else 16
NUM_MINIBATCHES = 200 if isFast else 1000000

# Ensure the training and test data is generated and available for this tutorial.
# We search in two locations for the prepared Berkeley Segmentation Dataset.
data_found = False

for data_dir in [os.path.join("data", "BerkeleySegmentationDataset")]:
    train_hr_path = os.path.join(data_dir, "train224")
    train_lr_path = os.path.join(data_dir, "train112")
    if os.path.exists(train_hr_path) and os.path.exists(train_lr_path):
        data_found = True
        break
        
if not data_found:
    raise ValueError("Please generate the data by completing the first part of this notebook.")
    
print("Data directory is {0}".format(data_dir))

#folders with training data (high and low resolution images) and paths to map files
training_folder_HR = os.path.join(data_dir, "train224")
training_folder_LR = os.path.join(data_dir, "train112")
MAP_FILE_Y = os.path.join(data_dir, "train224", "map.txt")
MAP_FILE_X = os.path.join(data_dir, "train112", "map.txt")

#image dimensions
NUM_CHANNELS = 3
LR_H, LR_W, HR_H, HR_W = 112, 112, 224, 224
LR_IMAGE_DIMS = (NUM_CHANNELS, LR_H, LR_W)
HR_IMAGE_DIMS = (NUM_CHANNELS, HR_H, HR_W)

Data directory is data/BerkeleySegmentationDataset


_unchecked_

下面定义了用于构造**图 6**中所提供的体系结构的函数。

In [27]:
#basic resnet block
def resblock_basic(inp, num_filters):
    c1 = C.layers.Convolution((3, 3), num_filters, init = C.he_normal(), pad = True, bias = False)(inp)
    c1 = C.layers.BatchNormalization(map_rank = 1)(c1)
    c1 = C.param_relu(C.Parameter(c1.shape, init = C.he_normal()), c1)
    
    c2 = C.layers.Convolution((3, 3), num_filters, init = C.he_normal(), pad = True, bias = False)(c1)
    c2 = C.layers.BatchNormalization(map_rank = 1)(c2)
    return inp + c2

def resblock_basic_stack(inp, num_stack_layers, num_filters):
    assert (num_stack_layers >= 0)
    l = inp
    for _ in range(num_stack_layers):
        l = resblock_basic(l, num_filters)
    return l

#SRResNet architecture
def SRResNet(h0):
    print('Generator inp shape: ', h0.shape)
    with C.layers.default_options(init = C.he_normal(), bias = False):
    
        h1 = C.layers.Convolution((9, 9), 64, pad = True)(h0)
        h1 = C.param_relu(C.Parameter(h1.shape, init = C.he_normal()), h1)
    
        h2 = resblock_basic_stack(h1, 16, 64)
    
        h3 = C.layers.Convolution((3, 3), 64, activation = None, pad = True)(h2)
        h3 = C.layers.BatchNormalization(map_rank = 1)(h3)
    
        h4 = h1 + h3
        ##here
    
        h5 = C.layers.ConvolutionTranspose2D((3, 3), 64, pad = True, strides = (2, 2), output_shape = (224, 224))(h4)
        h5 = C.param_relu(C.Parameter(h5.shape, init = C.he_normal()), h5)
    
        h6 = C.layers.Convolution((3, 3), 3, pad = True)(h5)

        return h6

_unchecked_

<p>我们遵循的文件和计算图是有点不同, 现在比以前的模型。不同的是在损失函数和学习率时间表。损失函数是 MSE, 学习率几乎是0.0001 的全部时间。我们在一开始就把它放得更高一些, 这可以帮助我们的模型在早期产生一些合理的结果。</p>

<p>训练与 isFast 设置为真实可能需要大约25分钟。</p>

In [28]:
def build_SRResNet_graph(lr_image_shape, hr_image_shape, net):
    inp_dynamic_axes = [C.Axis.default_batch_axis()]
    real_X = C.input(lr_image_shape, dynamic_axes=inp_dynamic_axes, name="real_X")
    real_Y = C.input(hr_image_shape, dynamic_axes=inp_dynamic_axes, name="real_Y")

    real_X_scaled = real_X/255
    real_Y_scaled = real_Y/255

    genG = net(real_X_scaled)
    
    G_loss = C.reduce_mean(C.square(real_Y_scaled - genG))
    
    G_optim = C.adam(G_loss.parameters,
                    lr = C.learning_rate_schedule([(1, 0.01), (1, 0.001), (98, 0.0001)], C.UnitType.minibatch, 10000),
                    momentum = C.momentum_schedule(0.9), gradient_clipping_threshold_per_sample = 1.0)

    G_G_trainer = C.Trainer(genG, (G_loss, None), G_optim)
    
    return (real_X, real_Y, genG, real_X_scaled, real_Y_scaled, G_optim, G_G_trainer)

In [29]:
SRResNet_model, real_X, real_X_scaled, real_Y, real_Y_scaled = train(SRResNet, LR_IMAGE_DIMS,
                                                                     HR_IMAGE_DIMS, build_SRResNet_graph)

Starting training
Generator inp shape:  (3, 112, 112)
 Minibatch[   1-  50]: loss = 0.071668 * 400;
 Minibatch[  51- 100]: loss = 0.010785 * 400;
 Minibatch[ 101- 150]: loss = 0.008225 * 400;
 Minibatch[ 151- 200]: loss = 0.006657 * 400;


_unchecked_

<p><b>图 7</b>显示了只有此模型的1000次迭代才能得到的结果。左边的图像是原始的 112 x 112 图像, 右边的是我们模型的输出。正如我们所看到的, 质量是远远不能接受的。此模型需要大约 10<sup>6</sup>迭代才能使质量变为实心。最后, 我们将展示我们可以通过足够多的迭代得到什么结果。</p>

In [30]:
#Figure 7
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/resnet_small_test.png",
      width = 390, height = 390)

_unchecked_

<p>此处创建的<b>SRResNet</b>模型是我们将尝试的最后一个模型的起始点: <b>SRGAN</b>。</p>

_unchecked_

<h2>SRGAN 超分辨率模型</h2>
<p><p>一个<b>GAN</b> (生成对抗性网络) 由两个相互监督的神经网络组成。甘斯首先介绍了伊恩古德费洛在纸<a href="https://arxiv.org/pdf/1406.2661.pdf">生成对抗性网</a>。</p>
<p>简单地说, 我们的甘, 像任何其他, 将由两部分组成:<ul>
<li><b>生成器网络:</b>我们的发电机网络将采取 112 x 112 图像和输出他们的超分辨版本的维度 224 x 224。我们的生成器网络的体系结构将与上面的<b>SRResNet</b>相同。</li>
<li><b>鉴别器网络:</b>我们的鉴别网络将在真正的 224 x 224 高分辨率的图像和那些由发电机网络产生的, 所以它可以学习区分一个与另一个。对于每 224 x 224 图像, 我们的鉴别器输出一个实数介于0和1之间, 它估计输入图像的概率是真实的 (而不是由发生器产生)。鉴别器的体系结构实质上是几个卷积层的序列, 每一个都跟随批规范化层, 漏 ReLU 层与α = 0.2。
<b>图 8</b>显示了确切的体系结构。它取自<a href="https://arxiv.org/pdf/1609.04802.pdf">SRGAN 文件</a>。<code>k</code>是接收字段大小, <code>n</code> 是层中的筛选器数, <code>s</code> 是步幅。</li>
</ul></p></p>

In [31]:
#Figure 8
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/discr_architecture.PNG")

_unchecked_

<p>在每次迭代中, 生成器都在尝试生成尽可能小的损耗值 (如在任何其他模型中)。这里的想法是将<b>对抗性损失</b>纳入到损失函数中, 这将考虑到鉴别器的意见。</p>

<p>同样, 在每次迭代中, 鉴别器都试图更好地分辨出真实的图像。</p>

<p>关键的想法是用以前训练过的<b>SRResNet</b>模型的权重初始化生成器网络权重, 然后开始培训。由于甘斯在训练过程中是不稳定的 (特别是在像超分辨率这样微妙的问题中), 这是一个好主意, 因为现在我们只需要稍微细化一下生成器, 所以我们不需要大量的参数变化和渐变, 希望不会变得非常大.因此, 可以说<b>SRGAN</b>是<b>SRResNet</b>的细化, 这 makse 意义。<p>现在, 我们可以根据所展示的体系结构来定义构造鉴别器的功能。</p>

In [32]:
#basic discriminator block
def conv_bn_lrelu(inp, filter_size, num_filters, strides = (1, 1), init = C.he_normal()):
    r = C.layers.Convolution(filter_size, num_filters, init = init, pad = True, strides = strides, bias = False)(inp)
    r = C.layers.BatchNormalization(map_rank = 1)(r)
    return C.param_relu(C.constant((np.ones(r.shape) * 0.2).astype(np.float32)), r)

#discriminator architecture
def discriminator(h0):
    print('Discriminator input shape: ', h0.shape)
    with C.layers.default_options(init = C.he_normal(), bias = False):
        h1 = C.layers.Convolution((3, 3), 64, pad = True)(h0)
        h1 = C.param_relu(C.constant((np.ones(h1.shape) * 0.2).astype(np.float32)), h1)
    
        h2 = conv_bn_lrelu(h1, (3, 3), 64, strides = (2, 2))
    
        h3 = conv_bn_lrelu(h2, (3, 3), 128)
        h4 = conv_bn_lrelu(h3, (3, 3), 128, strides = (2, 2))
    
        h5 = conv_bn_lrelu(h4, (3, 3), 256)
        h6 = conv_bn_lrelu(h5, (3, 3), 256, strides = (2, 2))
    
        h7 = conv_bn_lrelu(h6, (3, 3), 512)
        h8 = conv_bn_lrelu(h7, (3, 3), 512, strides = (2, 2))
    
        h9 = C.layers.Dense(1024)(h8)
        h10 = C.param_relu(C.constant(0.2, h9.shape), h9)
    
        h11 = C.layers.Dense(1, activation = C.sigmoid)(h10)
        return h11

_unchecked_

<h3>培训配置</h3>
<p><p>培训配置与<b>SRResNet</b>的配置相同, 但我们将使用较小数量的迭代, 因为我们只需要细化。其他路径和参数不变。我们把 minibatch 的大小, 以4在这里加快进程。较大的 minibatch 尺寸提供更精确的梯度近似和更好的结果。较小的 minibatch 尺寸提供更多的速度。</p></p>

In [33]:
#training configuration
MINIBATCH_SIZE = 2 if isFast else 4
NUM_MINIBATCHES = 200 if isFast else 100000

_unchecked_

<h3>计算图</h3>
<p><p>计算图现在与以前有很大的不同。发电机和鉴别器都有各自各自的损耗函数, 我们使用它们的信息来更新它们的参数。鉴别器显然需要在真实的 224 x 224 图像上和由发生器生成的映像上行动。启用此功能的标准方法是使用 <code>clone</code> 函数。有效的, 我们将有两个器, 一个是在真实图像上, 另一个在合成图像上。但是, 由于我们使用了 <code>clone</code> , 它们将共享参数。</p>
<p>最大的挑战是损失的功能。这是不容易创建一个损失的功能, 这将使我们的干得很好。系数需要仔细选择, 以确保我们的模型不分叉, 并开始产生无用的图像。对于损失函数的不同部分, 没有设置系数的一般规则, 这取决于我们正在处理的问题。找到好的系数配置通常是经过几次失败的尝试。</p>
<p>如果<math class="notranslate"> G </math>是我们的生成器和<math class="notranslate"> D </math>我们的鉴别符, 则在<a href="https://arxiv.org/pdf/1406.2661.pdf">生成对抗网</a>文件之后, <math class="notranslate"> G </math>和<math class="notranslate"> D </math>都在播放以下最小最大的游戏:</p></p>

In [34]:
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/min_max.PNG")

_unchecked_

<p>在这里, $x $ 的角色是由真正的 224 x 224 图像, 并 $z $ 的作用是由 112 x 112 的图像, 作为发电机的输入。因此, 对于合成图像, 鉴别器希望返回更大的数字 (接近 1), 用于实际的 224 x 224 图像和较小的数字 (接近 0)。相反, 生成器希望以这样的方式执行, 当在生成器的输出上进行评估时, 该鉴别机提供更大的输出 (接近于 1)。</p>

<p>对数可能有问题, 因为它们的渐变是无界的。无论该系数在发电机或鉴别器的对抗性损失面前有多小, 我们的梯度都有可能爆发。一旦他们这样做, 这是非常困难的模型返回, 以产生合理的图像。</p>

<p>为了说明这一点, 我们采用了<a href="https://arxiv.org/pdf/1703.10593.pdf">CycleGAN 纸</a>的想法, 其中对数损失被平方损失所取代, 如下所示。还要注意, 游戏现在是最大分钟, 而不是最小的最大值。</p>

In [35]:
Image(url = "https://superresolution.blob.core.windows.net/superresolutionresources/square_min_max.PNG")

_unchecked_

<p>除了对抗性和 MSE 损失外, 发电机的损耗函数也将具有<b>感知损耗</b>部分。引入感知损耗的一种常见方法是使用经过培训的<a href="http://www.robots.ox.ac.uk/~vgg/research/very_deep/">VGG19 网络</a>, 如<a href="https://arxiv.org/pdf/1609.04802.pdf">SRGAN 文件</a>中所述。</p>

<p>其思想是采用高分辨率图像及其生成的对应项, 并通过 <code>VGG19</code> 网络运行它们。这两种评价的平均平方差在层<b>relu5_4</b>是感性损失。显然, 引入这种损失可以帮助甘斯产生感知更令人满意的结果, 而不是只使用 MSE 损失 (当然是对抗性损失)。</p>

<p>现在, 我们准备定义一个函数, 它为<b>SRGAN</b>生成我们的计算图。发电机损耗系数是在几个文件的帮助下确定的, 试验和误差直到我们得到了可观的结果。通过尝试更多的价值, 我们可能会进一步改进, 并鼓励用户进行实验。</p>

In [36]:
#gan computation graph
def build_GAN_graph(genG, disc, VGG, real_X_scaled, real_Y_scaled, real_Y):  
    #discriminator on real images
    D_real = discriminator(real_Y_scaled)
    
    #discriminator on fake images
    D_fake = D_real.clone(method = 'share', substitutions = {real_Y_scaled.output: genG.output})
    
    #VGG on real images
    VGG_real = VGG.clone(method = 'share', substitutions = {VGG.arguments[0]: real_Y})
    
    #VGG on fake images
    VGG_fake = VGG.clone(method = 'share', substitutions = {VGG.arguments[0]: 255 * genG.output})
    
    #generator loss: GAN loss + MSE loss + perceptual (VGG) loss
    G_loss = -C.square(D_fake)*0.001 + C.reduce_mean(C.square(real_Y_scaled - genG)) + C.reduce_mean(C.square(VGG_real - VGG_fake))*0.08
    
    #discriminator loss: loss on real + los on fake images
    D_loss = C.square(1.0 - D_real) + C.square(D_fake)
    
    G_optim = C.adam(G_loss.parameters,
                    lr = C.learning_rate_schedule([(20, 0.0001), (20, 0.00001)], C.UnitType.minibatch, 5000),
                    momentum = C.momentum_schedule(0.9), gradient_clipping_threshold_per_sample = 0.1)
    
    D_optim = C.adam(D_loss.parameters,
                    lr = C.learning_rate_schedule([(20, 0.0001), (20, 0.00001)], C.UnitType.minibatch, 5000),
                    momentum = C.momentum_schedule(0.9), gradient_clipping_threshold_per_sample = 0.1)
    
    G_trainer = C.Trainer(genG, (G_loss, None), G_optim)
    
    D_trainer = C.Trainer(D_real, (D_loss, None), D_optim)
    
    return (G_trainer, D_trainer)

_unchecked_

<h3>培训模型</h3>
<p><p>由于我们必须对鉴别器和发生器进行交替更新, 所以现在的列车功能也大为不同。映射文件和图像维度与<b>SRResNet</b>模型中的大小相同。在培训之前, 我们将下载 pretrained <code>VGG19</code> 模型。可能需要几分钟。</p></p>

In [37]:
data_dir = os.path.join("data", "BerkeleySegmentationDataset")
if not os.path.exists(data_dir):
    data_dir = os.makedirs(data_dir)

models_dir = os.path.join(data_dir, "PretrainedModels")

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

print("Downloading VGG19 model...")
urlretrieve("https://www.cntk.ai/Models/Caffe_Converted/VGG19_ImageNet_Caffe.model",
            os.path.join(models_dir, "VGG19_ImageNet_Caffe.model"))
print("Done!")

Done!


In [38]:
def train_GAN(SRResNet_model, real_X, real_X_scaled, real_Y, real_Y_scaled):
    print("Starting training")

    reader_train_X = create_mb_source(MAP_FILE_X, LR_W, LR_H)
    reader_train_Y = create_mb_source(MAP_FILE_Y, HR_W, HR_H)
    
    VGG19 = C.load_model(os.path.join(models_dir, "VGG19_ImageNet_Caffe.model"))
    print("Loaded VGG19 model.")

    layer5_4 = VGG19.find_by_name('relu5_4')
    relu5_4  = C.combine([layer5_4.owner])
    
    G_trainer, D_trainer = build_GAN_graph(genG = SRResNet_model, disc = discriminator, VGG = relu5_4,
                                           real_X_scaled = real_X_scaled, real_Y_scaled = real_Y_scaled, real_Y = real_Y)
    
    print_frequency_mbsize = 50
   
    print("First row is discriminator loss, second row is generator loss:")
    pp_D = C.logging.ProgressPrinter(print_frequency_mbsize)
    pp_G = C.logging.ProgressPrinter(print_frequency_mbsize)

    inp_map_X = {real_X: reader_train_X.streams.features}
    inp_map_Y = {real_Y: reader_train_Y.streams.features}
    
    for train_step in range(NUM_MINIBATCHES):
        X_data = reader_train_X.next_minibatch(MINIBATCH_SIZE, inp_map_X)
        batch_inps_X = {real_X: X_data[real_X].data}
        
        Y_data = reader_train_Y.next_minibatch(MINIBATCH_SIZE, inp_map_Y)
        batch_inps_X_Y = {real_X: X_data[real_X].data, real_Y : Y_data[real_Y].data}
        
        D_trainer.train_minibatch(batch_inps_X_Y)
        pp_D.update_with_trainer(D_trainer)
        D_trainer_loss = D_trainer.previous_minibatch_loss_average

        G_trainer.train_minibatch(batch_inps_X_Y)
        pp_G.update_with_trainer(G_trainer)
        G_trainer_loss = G_trainer.previous_minibatch_loss_average
    
    model = G_trainer.model
    
    return model

_unchecked_

<p>请记住, 培训<b>SRGAN</b>的要求是<b>SRResNet</b>模型, 因此我们将其传递到训练函数中。</p>

<p>训练与 isFast 设置为真实可能需要大约20分钟。由于我们的<b>SRResNet</b> (用于初始化<b>SRGAN</b>) 和<b>SRGAN</b>都是经过少量迭代训练的, 因此<b>SRGAN</b>的结果可能不值得一看。我们不介意, 因为这仅仅是为了展示代码和培训程序和真正的模型 (培训足够的迭代) 在教程 CNTK 302 部分 A 中使用。</p>

In [39]:
SRGAN_model = train_GAN(SRResNet_model, real_X, real_X_scaled, real_Y, real_Y_scaled)

Starting training
Loaded VGG19 model.
Discriminator input shape:  (3, 224, 224)
First row is discriminator loss, second row is generator loss:
 Minibatch[   1-  50]: loss = 0.971728 * 100;
 Minibatch[   1-  50]: loss = 0.063828 * 100;
 Minibatch[  51- 100]: loss = 1.000000 * 100;
 Minibatch[  51- 100]: loss = 0.006320 * 100;
 Minibatch[ 101- 150]: loss = 1.000000 * 100;
 Minibatch[ 101- 150]: loss = 0.005728 * 100;
 Minibatch[ 151- 200]: loss = 1.000000 * 100;
 Minibatch[ 151- 200]: loss = 0.006376 * 100;


_unchecked_

现在, 这些模型可用于评估 CNTK 302 部分 A 中描述的任何图像。我们不建议使用这些模型, 因为它们经过了少量的迭代训练。我们建议使用预训练的模型, 并将其用于 CNTK 302 部分 a. 培训仅用于展示培训速度和程序。